In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob
import json
import os, sys
sys.path.append('..')
from utils import *
from random import randint

In [3]:
with open('../data/man.json') as fl:
    vs, adj = json.load(fl)

In [4]:
len(vs)

684

In [5]:
def sort_bydepth(vs, adj):
    roots = []
    for vert in vs:
        pointed = False
        for st, ls in adj.items():
            if vert in ls:
                pointed = True
                break
        if not pointed:
            roots.append(vert)
            
    print('Roots found:', len(roots))
    dmap = { v: 0 for v in roots }
    _, cadj = complete_graph(vs, adj)
    
    visited = {}
    queue = roots
#     print(roots)
    while len(queue):
        head = queue.pop(0)
        visited[head] = True
        
        for child in cadj[head]:
            if child not in visited:
                if child not in dmap:
                    dmap[child] = dmap[head] + 1
                queue.append(child)
#     print(len(dmap))
    sorted_vs = vs.copy()
    assert len(sorted_vs) == len(vs)
    sorted_vs = sorted(sorted_vs, key=lambda vert: dmap[vert])
    return sorted_vs, dmap
    
_vs, gdepth = sort_bydepth(vs, adj)
gobj = show_graph(_vs, adj, vdesc=lambda vert: str(gdepth[vert]))
gobj.render('depth')

Roots found: 33


'depth.jpg'

In [6]:
subgraphs = find_hops_2way(5, vs, adj, overlap=False)
len(subgraphs)

39

In [7]:
sorted_subgraphs = find_hops_2way(5, _vs, adj, overlap=False)
len(sorted_subgraphs)

47

In [11]:
from graphviz import Digraph
def render_subgraphs(tag, subgraphs, pretty=False):
    isRoot = {}
    dmap = {}
    groupmap = {}
    used = {}
    for gi, ((svs, sadj), subdepth) in enumerate(subgraphs):
        for vert, dp in subdepth.items():
            dmap[vert] = dp
        isRoot[svs[0]] = True
#         print(gi, len(svs))
        for vert in svs:
            groupmap[vert] = gi
    print('%s - %d/%d' % (tag, len(groupmap), len(vs)))

    def vdesc(vert):
        if vert in dmap:
            return str(dmap[vert])
        else:
            return ' '
        
    if not pretty:

        dot = Digraph(comment=vs[0], format='jpg')
        for vi, vert in enumerate(vs):
            if vert in groupmap:
                shape='doublecircle' if vert in isRoot else None
                cind = groupmap[vert] % 3
    #             print(vert, cind, groupmap[vert])
                clr = ['yellow', 'chartreuse', 'lightblue'][cind]
                dot.node(vert, vdesc(vert), shape=shape, color=clr)
            else:
                dot.node(vert, '   ')
            # dot.node(vert, vert, shape=shape)
        for vi, (vert, alist) in enumerate(adj.items()):
            for vto in alist:
                dot.edge(vert, vto)
        dot.render('grouped_%s' % tag)
    else:
        mang = Digraph(format='jpg', engine='dot', graph_attr={
            'splines':"ortho",
            'nodesep':'1',
            'rankdir':'LR',
            'fixedsize':'true',
         })
        
        for vi, vert in enumerate(vs):
            if vert in groupmap:
                shape='doublecircle' if vert in isRoot else None
                cind = groupmap[vert] % 3
    #             print(vert, cind, groupmap[vert])
                clr = ['yellow', 'chartreuse', 'lightblue'][cind]
                dot.node(vert, vdesc(vert), shape=shape, color=clr)
            else:
                dot.node(vert, '   ')
            # dot.node(vert, vert, shape=shape)
        for vi, (vert, alist) in enumerate(adj.items()):
            for vto in alist:
                dot.edge(vert, vto)
        dot.render('pretty_grouped_%s' % tag)
        

render_subgraphs('sorted', sorted_subgraphs)
render_subgraphs('random', subgraphs)

sorted - 532/684
random - 441/684


In [9]:
# s4 = find_hops_2way(4, _vs, adj, overlap=False)
# len(s4)

In [13]:
# render_subgraphs('sorted_h4', s4)

SyntaxError: invalid syntax (<ipython-input-13-bc47955bec36>, line 2)

In [21]:
for gi, ((svs, sadj), dmap) in enumerate(sorted_subgraphs):
    for hi in range(1, 6):
#         print(gi, hi, svs[0], len(svs))
        
        hvs = []
        for vert in svs:
            if dmap[vert] <= hi:
                hvs.append(vert)
        hvs = sorted(hvs, key=lambda vert: dmap[vert])
        __vs, __adj = subgraph(hvs, svs, sadj)

        with open('../data/graphs/%s_n%d.json' % (svs[0], hi), 'w') as fl:
            json.dump([
                __vs,
                __adj
            ], fl, indent=4)

### Don't forget to linearize the graph files

In [12]:
render_subgraphs('sorted', sorted_subgraphs, pretty=True)

sorted - 532/684


In [16]:
roots = []
for gi, ((svs, _), _) in enumerate(sorted_subgraphs):
    print(svs[0])
    roots.append(svs[0])
len(roots)

402528-404853
405057-403416
400454-403979
403492-403494
405393-401016
402620-403295
405186-402766
400732-403769
401628-403443
402505-402506
400612-400613
401091-401092
403144-403145
401149-401150
402003-402004
404824-401310
401248-404024
401069-401070
402646-402647
400007-403782
403894-403895
803117-402228
400108-400109
400468-400469
400360-400361
405005-402542
404947-405066
402517-402518
401045-401988
404245-404915
401259-401261
403110-403111
403155-405249
803162-404303
401027-404320
401051-401052
400032-400033
402835-404171
401271-401272
403167-403168
401333-401334
450123-400516
400044-400045
403821-403822
400861-400948
400330-400331
401360-401361


47

In [17]:
with open('../data/subset.txt', 'w') as fl:
    fl.write('\n'.join(roots))